# Model Template
This notebook aggregates all the information from the previous two notebooks in a more concise way, using data that directly mimics the final collected data in scale and dimensionality. A visualization method is implemented for easier exploration of both the model and the generated or collected data, even for higher dimensionsionalities.
One yet unexplored detail is that we are looking to resolve forces on each of the components, not only the external forces. Resolving these forces allows us, through physics calcualtions, to resolve the load state on the suspension as a whole.

## Imports
Import necessary libraries

In [ ]:
## Basic imports
import numpy as np
from mpl_toolkits import mplot3d
import pandas as pd
import plotly.express as px
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.base import BaseEstimator, TransformerMixin, clone
from scipy.interpolate import griddata

## Imports for polynomial
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import MultiTaskElasticNet, MultiTaskElasticNetCV
from sklearn.pipeline import Pipeline

## Imports for displaying
import ipywidgets as widgets
from IPython.display import display

## colab's version of plotly is too old for something we want to do. In the future this will probably be unecessary ONLY UNCOMMENT IF NEEDED
!pip install plotly==4.9.0
import plotly as py
import plotly.graph_objects as go

## Display
To make it easier to explore the data, we implement a widget that allows us to choose plotting variables and what type of data we want to present (raw data, model predictions or both).

In [ ]:
## This takes the raw data and model, and given the user inputs, plots 1 variable against two others.
## It can plot both the raw data, the model's predictions based on the raw data 
## Or both.

def plot_data(raw_dataframe,predicted_dataframe, x_axis, y_axis, z_axis_raw, z_axis_pred, z_datatype,error_heatmap, model = None):
  ##In orange are the predictions form the model. In blue, the original values.

  z_error = np.abs(raw_dataframe[z_axis_raw] - predicted_dataframe[z_axis_pred])

  if z_datatype == 'raw':
    ## plot the raw data
    fig = px.scatter_3d(raw_dataframe, x = x_axis, y=y_axis, z = z_axis_raw)
    fig.update_traces(marker=dict(size=2))
  
  if z_datatype == 'prediction':
    #plot the predicted data
    fig = px.scatter_3d(predicted_dataframe, x = raw_dataframe[x_axis], y=raw_dataframe[y_axis], z = z_axis_pred)
    fig.update_traces(marker = dict(color = "orange"))
    fig.update_traces(marker=dict(size=2))

  if z_datatype == 'both':
    fig = px.scatter_3d(raw_dataframe, x = x_axis, y=y_axis, z = z_axis_raw)
    fig.add_scatter3d(x = raw_dataframe[x_axis], y=raw_dataframe[y_axis], z = predicted_dataframe[z_axis_pred], mode = 'markers',marker = dict(color = "orange"))
    fig.update_traces(marker=dict(size=2))

  if z_datatype == 'error':
    fig = px.scatter_3d(raw_dataframe, x = x_axis, y=y_axis, z = z_error, color=z_error)
    fig.update_traces(mode = 'markers', marker=dict(size=2))

  if error_heatmap == True:
    fig.add_scatter3d(x = raw_dataframe[x_axis], y=raw_dataframe[y_axis], z = raw_dataframe[z_axis_raw], mode = 'markers', 
                      marker=dict(color=z_error,colorscale = 'Jet',opacity = 0.25,size=5))
    print(np.max(z_error), np.min(z_error), np.mean(z_error))

  
  fig.show()

def choose_plot_data(raw_data=None, predicted_data=None, trained_model = None):
  ## takes a raw_data dictionary and transforms it into a dataframe
  #print(raw_data['upper'])
  #print(predicted_data['upper'])
  dict_temp = {}
  max_len = 0
  for key in raw_data.keys():
    for i, list_of_data in enumerate(raw_data[key]):
      for j, data in enumerate(list_of_data.T):
        dict_temp[key+"_"+str(i)+"_"+str(j)] = data.copy()
        if len(data) > max_len:
          max_len = len(data)
  for key in dict_temp.keys():
    if len(dict_temp[key])< max_len:
      concat_array = np.zeros((max_len-len(dict_temp[key])))
      concat_array[:] = np.nan
      dict_temp[key] = np.concatenate((dict_temp[key], concat_array))
    


  raw_dataframe = pd.DataFrame(dict_temp)
  opts_raw = raw_dataframe.columns #gets the axis that can be plotted

  if predicted_data is not(None):
    ## takes a predicted_data dictionary and transforms it into a dataframe
    dict_temp = {}
    for key in predicted_data.keys():
      for i in range(predicted_data[key].shape[1]):
          dict_temp[key+"_0_"+str(i)+"_pred"] = predicted_data[key][:, i].copy()
          if len(data) > max_len:
            max_len = len(predicted_data[key][:, i])
    for key in dict_temp.keys():
      if len(dict_temp[key])< max_len:
        concat_array = np.zeros((max_len-len(dict_temp[key])))
        concat_array[:] = np.nan
        dict_temp[key] = np.concatenate((dict_temp[key], concat_array))

    predicted_dataframe = pd.DataFrame(dict_temp)
    opts_pred = predicted_dataframe.columns #gets the axis that can be plotted
  else:
    opts_pred = "None"
    predicted_dataframe = None


  datatype_opts = ['raw', 'prediction', 'both', 'error'] #set the types of possible plots

  ## create the widget
  w = widgets.interactive(plot_data, 
                          raw_dataframe = widgets.fixed(raw_dataframe), 
                          predicted_dataframe = widgets.fixed(predicted_dataframe),
                          x_axis = opts_raw, y_axis = opts_raw, z_axis_raw  = opts_raw, z_axis_pred = opts_pred,
                          z_datatype = datatype_opts, 
                          error_heatmap = True,
                          model = widgets.fixed(trained_model))

  ## display the widget
  display(w)

## Preprocessing
This section deals with, given the collected data, create a dataset that can be succesfully fed into the model we have. The model expects the data fed into it to be a `pandas.DataFrame` for **each** of the individual parts suspension parts we collect data from.

### Front Suspension
The front suspension has 3 main parts, which are sensitive to forces, which will be measured with a minimum of 6 gauges. The forces and gauges are thus distributed as follows:
 - Upper Front Arm: 2 forces, 2 gauges
 - Lower Front Arm: 3 forces, 3 gauges
 - Tie Rod: 1 force, 1 gauge

In [ ]:
## code cell to preprocess the front suspension data once we have it

### Rear Suspension
The rear suspension has 2 main parts, which are sensitive to forces and moments, which will be measured with a minimum of 7 gauges. The forces and gauges are thus distributed as follows:
 - Rear Lower Arm: 3 forces, 3 gauges
 - Rear Upper Arm: 2 forces, 2 moments, 4 gauges

In [ ]:
## code cell to preprocess the rear suspension data once we have it

## Data Pipeline
Now that it has been established that the calibration is going to be performed with individual tests on each of the parts, it makes more sense to create a data pipeline that is comprised of several models, one for each part of the suspension. This helps in the following ways:
 - Each model is simpler. The most complex model is a 4 input to 4 output model, and the simplest one is one input to one output. 
 - There is no need to worry about how to handle missing data from one calibration process to the next. We can treat each of the parts as making a "modular" prediction, independent of the rest of them.
 - Less data required: The simpler the model, the less data required to capture its behaviour. This gives us finer control over what parts need more testing and less testing in the real world.
 - Having a pipeline allows us to train the entire thing at once and make overall changes at once instead of doing it one model at a time. It also allows us to save the model as a single file instead of a bunch of individual models

First we need to implement an estimator that can independently make predictions for each of the "modules" we want. The `multioutputregressor` we were using before is not an option because that is strictly for when we want a model for each of the output **variables**, but we want a model for each of the output **datasets** instead. The standard pipeline object is also not going to help us here beyond the very last step because it can only implement a single estimator. What I propose is to create a class that can generate many estimators and access them freely, and that knows, given an input dataset and information about how many forces and gauges each module has, what the inputs and outputs of that dataset are. This class uses the polynomial pipeline we created previously. An important restriction to this class is that it should "[be importable and live in the same module as when the object was stored.](https://docs.python.org/3.4/library/pickle.html)" to ensure that our final trained multi model class is pickleable (able to be stored for future use). I am not entirely sure what that means but I'll assume that by having the class definition in this notebook it would be enough to allow it to be pickled.

The type of dataset that is going to be used is a dictionary. This allows us to have different lengths of datasets for each of the calibration parts and makes it easy to  handle the data. This was chosen instead of a DataFrame mainly to avoid having to deal with the $NaN$ values that would have to be introduced in the dataframe because of the different amounts of data in each of the calibration tests. It can still be directly converted into a DataFrame whenever needed for visualization.

In [ ]:
## creating the class to generate and train models

class modular_polynomial_model():
  def __init__(self, data_dict):

    self.data_dict = data_dict.copy()
    self.dict_keys = list(self.data_dict.keys())
    self.model_list = []

  def generate(self):
    ## generates the models and store them in the same dictionary
    
    poly_model = Pipeline([('poly', PolynomialFeatures(degree=4)),
                  ('multi_elastic', MultiTaskElasticNet())])                     #generates a pipeline instance with the model to be used

    for key in self.dict_keys:
      self.model_list += [clone(poly_model)]                                     #append as many models as necessary for the overall model

  def fit(self):
    ## fits all the model with the appropriate data
      for i, key in enumerate(self.dict_keys):
        loads, voltages = self.data_dict[key][0], self.data_dict[key][1]
        self.model_list[i].fit(voltages, loads)                                  #from voltages, fit to predict loads

  def predict(self, inference_data_dict):
    ## predicts all the results using the supplied data and the models
    
    predictions = {}

    for i, key in enumerate(self.dict_keys):
      _, voltages = inference_data_dict[key][0], inference_data_dict[key][1]
      predictions[key] = self.model_list[i].predict(voltages)

    return predictions

## Demonstration

In this section we show how everything ties together

### *TEMPORARY* - Data Generation
While we have no real world data, the synthetic data will be generated and formated here. The following are assumptions on the data:
- Load data varies in the range $[-5000, 5000]$. This makes no distinction between moments and forces, however there doesn't seem to be any advantage in differentiating the two for synthetic data, as all it would require would be to add up gauge responses due to forces and moments.
- Force data is uniformly distributed in the space (non sparse)
- Gauge readings in volts are centered around $3.3V$ and have variation in the order of volts, with precision in the milivolts
  - One thing that I noticed is that with the default parameters the model has trouble to deal with data if the variation itself is in the order of milivolts. This can either be because of computater precision problems or because the model has a hard time dealing with transforming very small variations into big variations. With a variation in the order of 1 volt however, it had no problem fitting the data. Looking at past gauge data, it does seem to me that the data varies more than in the milivolts, so for now I'm going to simply assume that these are the voltage variations we are also going to see. If this turns out to not be the case, it is necessary to either optimize the model for this type of data or amplify the variation prior to feeding it to the model
- Gauge readings are non linear with force distribution, but not by much. Just a slight curvature.
- Noise is present, with standard deviation of about $2.5\%$ of the gauge milivolt variation (meaning that $96\%$ of noise is going to fall at a $5\%$ difference from the voltage variation)
- In the real world tests we are striving to generate data in such a way that each of the gauges is highly sensitive to a single load. Assuming we are succesful at that, this translates to a generation function where the main diagonal on the linear transformation has higher values than the off-diagonal values. The way we are going to do this is:
  - $|Main\ diagonal\ values)| >=0.8$
  - $|Off\ diagonal\ values| <=0.15$

##### Data Generation Function

In [ ]:
## The basic linear vector can be therefore written as a
## class which has the weights kij either randomly generated between 0 and 1 (default behaviour)
## or provided by the user. The class also is initialized with some other standard
## values, which can all be changed for testing.


class Gauge:
  def __init__(self, force_range = [-1,1], n_forces = 6, n_gauges = 6, weights = np.array(0), sin_params = np.array(0), default_gauge_voltage = 3.3, voltage_variation_magnitude = 0.0001):
    self.force_range = force_range
    self.n_forces = n_forces
    self.n_gauges = n_gauges
    self.default_gauge_voltage = default_gauge_voltage
    self.voltage_variation_magnitude = voltage_variation_magnitude

    if sin_params.shape != (): #check if sinusoidal parameters are default, if not default, check if they are the correct number of parameters
      assert sin_params.shape[0] == n_forces, f'the number of sinusoidal parameters ({sin_params.shape[0]}) doesnt match the number of forces ({n_forces})'
    self.sin_params = sin_params
    
    if not weights.shape:
      weights = np.zeros((n_gauges, n_forces))
      for i, row in enumerate(weights):
        for j, value in enumerate(row):
          if i == j:
            val_temp = np.random.uniform(0.8,1)*np.random.choice([-1,1])
          else: 
            val_temp = np.random.uniform(-0.15,0.15)
          weights[i,j] += val_temp
          #print(i,j, value)
      #weights = np.random.rand(n_gauges, n_forces)

    assert weights.shape[1] == n_forces, f'the number of forces is {n_forces} and weights only combines {weights.shape[1]} forces'
    assert n_gauges == weights.shape[0], f'the number of gauges ({n_gauges}) doesnt match the length of the weights array ({weights.shape[0]})'
    self.weights = weights*voltage_variation_magnitude

  def data_generator(self, n_datapoints = 10, gauge_type = 'linear', force_distribution = 'uniform', noisy = False): 
                                          #this method returns as many gauge voltages sets as required (default is 10 datapoints)
                                          #the type can be modified to have its behaviour changed according to whatever test
                                          #we want to run (default is linear non noisy randomly uniformly distributed)
    voltage_list = []
    force_list = []
    max_voltage = 0

    for i in range(n_datapoints):

      if force_distribution is 'uniform':
        force_temp = np.random.uniform(*self.force_range, size = self.n_forces) #generates the force data
      
      if force_distribution is 'axes':
        axis = np.random.randint(0,self.n_forces) #choose one axis
        force_temp = np.zeros(shape = (self.n_forces)) #create a vector of zeros
        force_temp[axis] = np.random.uniform(*self.force_range) #fill one of the axis with the correct range of forces

      if gauge_type is 'linear':
        voltage_temp = np.matmul(self.weights, force_temp) #generates the voltage data for a linear gauge

      if gauge_type is 'non_linear':
        force_nonlinear_temp = force_temp * np.sin((self.sin_params[:,0]*np.pi*force_temp/self.force_range[1]) + self.sin_params[:,1]) #creates the non linear "term"
        voltage_temp = np.matmul(self.weights, force_nonlinear_temp) #generates the voltage data for a non linear gauge

      
      if np.max(np.abs(voltage_temp)) > max_voltage: #updates the max voltage variation seen so far
          max_voltage = np.max(np.abs(voltage_temp))

      if noisy: #if the gauge is noisy, add noise to the data. 
                #by arbitrary decision, noise is normally distributed with standard deviation of 10% of the 
                #max voltage variation seen so far
        
        standard_dev = 0.025*max_voltage
        noise = np.random.normal(loc = 0, scale = standard_dev, size = voltage_temp.shape) #generates a noise vector
        
        voltage_temp += noise

      voltage_list.append(voltage_temp)
      force_list.append(force_temp)


    ## now we scale the gauge values to the correct values
    voltage_list = np.array(voltage_list)+ self.default_gauge_voltage

    return np.array(force_list), np.array(voltage_list)


##### Data generation

In [ ]:
## TEMPORARY: this section uses data generation to 
np.random.seed()

## Front Upper
front_upper_sin_params = np.array(np.array([[0.15,1],[0.15, 2]])) #choose the sinusoidal parameters
front_upper = Gauge(force_range = [-5000,5000], n_forces = 2, n_gauges = 2, sin_params = front_upper_sin_params) ## creating a gauge object for the upper front A arm
front_upper_forces, front_upper_voltages = front_upper.data_generator(n_datapoints = 25, gauge_type = 'non_linear', noisy = True, force_distribution='uniform')  #generates 2 sets of data, one with the forces and one with the voltage responses
front_upper_forces_val, front_upper_voltages_val = front_upper.data_generator(n_datapoints = 300, gauge_type = 'non_linear', noisy = False, force_distribution='uniform')  #generates 2 sets of data, one with the forces and one with the voltage responses
front_upper_data = np.concatenate((front_upper_forces, front_upper_voltages),axis = 1)

print(f'front uppper (n_samples, n_forces + n_voltages) = {front_upper_data.shape}')

## Front Lower
front_lower_sin_params = np.array(np.array([[0.1,1],[0.15, 2], [0.12, 1.5]])) #choose the sinusoidal parameters
front_lower = Gauge(force_range = [-5000,5000], n_forces = 3, n_gauges = 3, sin_params = front_lower_sin_params) ## creating a gauge object for the lower front A arm
front_lower_forces, front_lower_voltages = front_lower.data_generator(n_datapoints = 25, gauge_type = 'non_linear', noisy = True, force_distribution='uniform')  #generates 2 sets of data, one with the forces and one with the voltage responses
front_lower_forces_val, front_lower_voltages_val = front_lower.data_generator(n_datapoints = 300, gauge_type = 'non_linear', noisy = False, force_distribution='uniform')  #generates 2 sets of data, one with the forces and one with the voltage responses
front_lower_data = np.concatenate((front_lower_forces, front_lower_voltages),axis = 1)

print(f'front lower (n_samples, n_forces + n_voltages) = {front_lower_data.shape}')

## Tie Rod
tierod_sin_params = np.array(np.array([[0.13,1.5]])) #choose the sinusoidal parameters
tierod = Gauge(force_range = [-5000,5000], n_forces = 1, n_gauges = 1, sin_params = tierod_sin_params) ## creating a gauge object for the tie rod
tierod_forces, tierod_voltages = tierod.data_generator(n_datapoints = 10, gauge_type = 'non_linear', noisy = True, force_distribution='uniform')  #generates 2 sets of data, one with the forces and one with the voltage responses
tierod_forces_val, tierod_voltages_val = tierod.data_generator(n_datapoints = 300, gauge_type = 'non_linear', noisy = False, force_distribution='uniform')  #generates 2 sets of data, one with the forces and one with the voltage responses
tierod_data = np.concatenate((tierod_forces, tierod_voltages),axis = 1)

print(f'tierod (n_samples, n_forces + n_voltages) = {tierod_data.shape}')

## Rear Lower
rear_lower_sin_params = np.array(np.array([[0.1,1],[0.11, 1.5], [0.15, 2]])) #choose the sinusoidal parameters
rear_lower = Gauge(force_range = [-5000,5000], n_forces = 3, n_gauges = 3, sin_params = rear_lower_sin_params) ## creating a gauge object for the lower front A arm
rear_lower_forces, rear_lower_voltages = rear_lower.data_generator(n_datapoints = 25, gauge_type = 'non_linear', noisy = True, force_distribution='uniform')  #generates 2 sets of data, one with the forces and one with the voltage responses
rear_lower_forces_val, rear_lower_voltages_val = rear_lower.data_generator(n_datapoints = 300, gauge_type = 'non_linear', noisy = False, force_distribution='uniform')  #generates 2 sets of data, one with the forces and one with the voltage responses
rear_lower_data = np.concatenate((rear_lower_forces, rear_lower_voltages),axis = 1)

print(f'rear lower (n_samples, n_forces + n_voltages) = {rear_lower_data.shape}')

## Rear Upper
rear_upper_sin_params = np.array(np.array([[0.1,1],[0.1, 0.9], [0.12, 2.3], [0.05, 1.2]])) #choose the sinusoidal parameters
rear_upper = Gauge(force_range = [-5000,5000], n_forces = 4, n_gauges = 4, sin_params = rear_upper_sin_params) ## creating a gauge object for the lower front A arm
rear_upper_forces, rear_upper_voltages = rear_upper.data_generator(n_datapoints = 30, gauge_type = 'non_linear', noisy = True, force_distribution='uniform')  #generates 2 sets of data, one with the forces and one with the voltage responses
rear_upper_forces_val, rear_upper_voltages_val = rear_upper.data_generator(n_datapoints = 300, gauge_type = 'non_linear', noisy = False, force_distribution='uniform')  #generates 2 sets of data, one with the forces and one with the voltage responses
rear_upper_data = np.concatenate((rear_upper_forces, rear_upper_voltages),axis = 1)

print(f'rear upper (n_samples, n_forces + n_voltages) = {rear_upper_data.shape}')

front uppper (n_samples, n_forces + n_voltages) = (25, 4)
front lower (n_samples, n_forces + n_voltages) = (25, 6)
tierod (n_samples, n_forces + n_voltages) = (10, 2)
rear lower (n_samples, n_forces + n_voltages) = (25, 6)
rear upper (n_samples, n_forces + n_voltages) = (30, 8)


#### Data Formatting
Once we have all the data, we create dictionaries to handle it

In [ ]:
## creating a dictionary for each of the suspension types

## Front suspension
front_dict = {"upper": [front_upper_forces, front_upper_voltages],
              "lower": [front_lower_forces, front_lower_voltages],
              "tierod":[tierod_forces, tierod_voltages]
}

front_dict_val = {"upper": [front_upper_forces_val, front_upper_voltages_val],
              "lower": [front_lower_forces_val, front_lower_voltages_val],
              "tierod":[tierod_forces_val, tierod_voltages_val]
}

## Rear suspension
rear_dict = {"upper": [rear_upper_forces, rear_upper_voltages],
              "lower": [rear_lower_forces, rear_lower_voltages]
}

rear_dict_val = {"upper": [rear_upper_forces_val, rear_upper_voltages_val],
              "lower": [rear_lower_forces_val, rear_lower_voltages_val]
}

#### Model Training
Using the dictionaries just created, we train the model using the `modular_polynomial_model()` class we created and its methods(functions)

In [ ]:
## front suspension
front_model = modular_polynomial_model(front_dict) #initializes the class with one of the dictionaries
front_model.generate()                             #generates every one of the models used for each part of the suspension
front_model.fit()                                         #fits each of the models to its corresponding data
front_dict_pred = front_model.predict(front_dict_val)         #make predictions using this model and store the results in a variable

##rear suspension
rear_model = modular_polynomial_model(rear_dict) #initializes the class with one of the dictionaries
rear_model.generate()                             #generates every one of the models used for each part of the suspension
rear_model.fit()                                         #fits each of the models to its corresponding data
rear_dict_pred = rear_model.predict(rear_dict_val)         #make predictions using this model and store the results in a variable

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1373726.709100143, tolerance: 46326.20537315866

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1284764.0314962803, tolerance: 65096.59755109597

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 4736758.034635489, tolerance: 88461.61700508295

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2046136.5145854913, tolerance: 56933

#### Displaying the results
Using the functions we created, displaying this dictionary data is easy and allows for quick visual checking. Because of the way I formatted my dictionaries, the variables are written in the following format `(suspension_part)_(load[0]/gauge[1])_(variable_number)`, and the variable number depends on the amount of that type of variable. For instance, if we have 3 gauges for the front lower, the numbers can be either `0`, `1` or `2`. Knowing which one is which depends on the preprocessing steps.

##### Front

In [ ]:
## Front suspension
choose_plot_data(raw_data = front_dict_val, predicted_data = front_dict_pred, trained_model=front_model)

interactive(children=(Dropdown(description='x_axis', options=('upper_0_0', 'upper_0_1', 'upper_1_0', 'upper_1_…

##### Rear

In [ ]:
## Rear suspension
choose_plot_data(raw_data = rear_dict_val, predicted_data = rear_dict_pred, trained_model=rear_model)

interactive(children=(Dropdown(description='x_axis', options=('upper_0_0', 'upper_0_1', 'upper_0_2', 'upper_0_…

## Number of Real World Datapoints
- Having this error estimate more easily available and viewable, we need to explore how much data do we need for a good fit for each of the models and their dimensionality.
  - The criteria to decide this will be an iterative process in which the number of datapoints for each of the parts is going to be changed until the max error is equal to roughly 1500 "load". The number of points for this error are going to be recorded. This will give us an estimate of the relative amount of data for each of the parts we are going to need for a certain accuracy level, and thus we can scale our tests accordingly depending on the total amount of data collection runs we intend to do.
    - Tierod: No lower bound, except the minimum for the order of polynomial used. If it is relatively linear even as little as $10$ datapoints are enough. Still need a few just to make sure it is in fact relatively linear.
    - Front Upper: Roughly $25$ datapoints for max error of 1900 and average of 500. 
    - Front Lower: Roughly $25$ datapoints for max error of 2000 and average of 700.
    - Rear Upper: Roughly $30$ datapoints for max error of 2000 and average of 500.
    - Rear Lower: Roughly $25$  datapoints for max error of 1500 and average of 500.

### Conclusion
After extensive testing, the most determinant factor in choosing the amount of points is how linear the function is. The dimensionality does not seem to have a big effect on the necessary number of datapoints. The tierod can have very few datapoints, and it would be benefitial to perform a few extra tests on the rear upper compared to the other parts. I recommend at least 120 total datapoints, if possible, however if we are convinced the behaviour is very linear, going as low as 70 could be an option. A recommended ratio of total datapoint could be as follows:
 - Tierod:10%
 - Front Upper/Lower and rear lower: 21% each
 - Rear Upper: 27%